# Lab 6 - Exploring Azure

## Welcome

This is it. This is our final lab together and one that is entirely unique in terms of approach and expectations. I'll explain all of that in a moment, but first I wanted to congratulate all of you.

#### Well done.

This course - _this quarter_ - is a difficult one and all of you have put in the time and effort to get through it. For some of you, python might have clicked immediately; for others, it can take time... _it can still be happening_. But, just like with writing, the more you do it, the better you'll become.

As I've stressed repeatedly, rather than memorizing every piece of syntax, make sure you focus on **how** computation works. The core ideas of flow control, iteration, abstraction... functions, classes, and methods. These ideas aren't unique to python and if you can grasp them; then, you're able to work in almost any programming language. **Yes**, there will be hiccups, but if you can _think computationally_ if you can break apart a big problem into the steps that a computer can follow - then you can accomplish amazing things.

#### Which brings us back to this lab.

Unlike in your other labs, everything in this lab is 'new.' We've never had the ability to access the Azure services at this scale before and what you're learning here really is a skillset that's at the forefront of how geospatial technologies are moving. We're going to be working directly with REST APIs and virtual machines, intergrating our work flow into the computational power available through cloud services.

But, that opportunity comes with a cost. Unlike in our other labs, I haven't necessarily **done** this. In the past, a unique error might pop up (as we are all running complicated systems that leverage lots of different libraries), but fundamentally I knew what the solution _should be_ every time. Here, that isn't _necessarily_ the case. We're going to be working through this **together**. 

It's a risk, but it comes with an awfully big reward.
#### So what does that mean?
Well, it means that grade wise you are all going to do fine. Let's just get that out of the way. It means that we have to trust one another and help one another through this process.

And, with all of that in mind, let's begin.

## Getting set up.

As always, you're going to want to build your environment.
I would recommend you use python 3.7 and that you install jupyter, folium, and geopandas. I used the `-c conda-forge` to install all three at once, but do what works for you.

In addition, you're going to need to install the azure services we're going to be using. 
`pip install azure-cognitiveservices-search-newssearch`

You'll also have access to their maps service and their sentiment analysis (although you could also use one of our other mapping, geocoding, or sentiment analysis services instead - such as folium, geopy, or NLTK).

## Our goal and some guidance

Here's where things get tricky. Unlike in other labs, I don't have a set of questions for you to work through. Instead, I'm going to give you an end goal and then some tips on how you might approach it. I'll also be constantly visiting each group to help you generate ideas, goals, and methods.

For what it's worth, this is how your Cartography seminar will work, so think of this as a bit of a taste of what's to come. **Have fun**. Be bold and be creative. 

### Your task: Create a map of how people are talking about a place
**Wait, what?**

What I want you to do is create a map that visualizes how a place is being talked about in the news, on social media, etc.

**Wait, how?**

Well, take a look at [this quickstart guide for sentiment analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quickstarts/python). You can find another [tutorial here](https://www.pingshiuanchua.com/blog/post/simple-sentiment-analysis-python). That second one uses a few approaches, but I recommend you stick with azure or NLTK. 

Sentiment analysis gives you a very rough, machine-learning creates sense of the 'emotion' (positive or negative) in a given set of texts. It's extremely useful, for example, if you are trying to monitor how people is talking about your business(es) online. 

Then take a look at [this guide on conducting news searches](https://docs.microsoft.com/en-us/azure/cognitive-services/bing-news-search/news-sdk-python-quickstart). (note: you've already set up your environment as they do at the beginning). 

You can also read about azure maps' geocoding service [here](https://docs.microsoft.com/en-us/rest/api/maps/search).

But, remember, you have other sources of information as well! You could - for example - monitor twitter for tweets about a place; you could use the geopy geocoder; you could - being careful with your credits - use the Arc GIS API geocoder or other services.

**At this point, you have a lot of tools open to you**. Talk with your group, talk with me, come up with a plan.

### A sample workflow

If you're stuck, you can follow along with me as I (try to) biuld something during the lab.

Here's the workflow I'll follow:
1. Conduct a news search about an area (probably Tacoma, but if that doesn't get enough results somewhere else).
2. Run the results of that search through the sentiment analysis
3. Map it
        a. First, just simply putting points on a map with the color related to the sentiment.
        b. Then, getting fancier. I'll try to create a heat map and then even normalize that heat map based on sentiment.
        
        
You can find the keys for the existing services in a password protected file [here](https://github.com/UWTMGIS/GIS501_w19_files/blob/master/w19_azurekeys.pdf.zip). I will provide the password in class.
    

# Let's go!
## Get fancy, get creative, have fun.

In [103]:
from azure.cognitiveservices.search.newssearch import NewsSearchAPI
from msrest.authentication import CognitiveServicesCredentials
subscription_key = '5539bb45ad484b06941f3237b50d3ca6'
search_term = ['Game of Thrones']

client = NewsSearchAPI(CognitiveServicesCredentials(subscription_key))
news_result = client.news.search(query=search_term, market="en-us", count=50)

newstext = []
newssource = []
location = []

if news_result.value:
    for news in news_result.value:
        #print('News name: {}'.format(news.description))
        #print()
        newstext.append(str(news.description))
        newssource.append(str(news.provider[0].name))
for i in range(0, len(news_result.value)):
    location.append(BizLoc(newssource[i]))
    
    '''print(newstext)
else:
    print('No news data..')'''

import requests, json

def BizLoc(name):
    place = 'https://atlas.microsoft.com/search/poi/json?subscription-key=UiyHPio-CFlojmpNinmqMjJ2BUbauzsgcWjIriCHxY4&api-version=1.0&query='+str(name)+'&limit=1'
    response = requests.get(place)
    stuff = response.json()
    if stuff['results']:
        x, y = stuff['results'][0]['position']['lon'], stuff['results'][0]['position']['lat']
    else:
        x, y = 0,0
    return x,y

import pandas as pd
def sentiment(text):
    import requests, json
    documents = {'documents': [
        {'id': 1, 'text': text}
    ]}
    azure_key = 'fc7f9e002a924d7db06e7f23bacafb62'
    azure_endpoint = 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0'
    assert azure_key
    sentiment_azure = azure_endpoint + '/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key' : azure_key}
    response = requests.post(sentiment_azure, headers=headers, json=documents)
    sentiments = response.json()
    return sentiments

azure_results = [sentiment(text) for text in newstext]
azure_score = [row['documents'][0]['score'] for row in azure_results]
azure = list(zip(newssource, location, newstext, azure_score))
columns = ['source', 'location', 'text', 'score']
azure_df = pd.DataFrame(azure, columns = columns)

azure_df.head()
#print(azure_df)

,source,location,text,score
0,USA Today,"(-22.95471, -43.1739)",Game of Thrones' Season 8 character posters HB...,0.748307
1,Business Insider,"(40.73965, -73.99104)","At $272, the bed isn't exactly cheap, but then...",0.953299
2,Los Angeles Times,"(29.76294, -95.36465)","In a major television industry shakeup, HBO Ch...",0.500000
3,International Business Times,"(51.50492, -0.01867)",“Game of Thrones” fans may have been excited w...,0.500000
4,digitaltrends.com,"(31.84494, -99.408)",Game of Thrones might well be one of the most ...,0.857902


In [118]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas as pd, folium, geopandas
from folium.plugins import MarkerCluster

coords = zip(azure_df['location'])

#I am going to collapse a for loop into a single line here, watch how

geometry = [shapely.geometry.Point(c) for c in coords]
#Shapely is a library with the point data type GeoPandas uses for coordinates

#Now, I'm going to make that geometry into a GeoSeries
geolist = geopandas.GeoSeries(geometry)

#Ok, let's put it all together.
geoPoints = geopandas.GeoDataFrame(
        azure_df,
        geometry=geolist)

#And now, TO A WEB MAP
geoPoints.crs=fiona.crs.from_epsg(4236) #What am I doing here? Why?

winteriscoming = folium.Map(
                location=[33, -23],
                zoom_start=2,
                tiles='Stamen Terrain')

#starkpoints = folium.features.GeoJson(geoPoints.to_json()).add_to(winteriscoming)

cluster = MarkerCluster()

for i in range(0, len(azure_df)):
    cluster.add_child(folium.Marker(location=[azure_df.iloc[i]['location'][1], azure_df.iloc[i]['location'][0]]
                    , popup=azure_df.iloc[i]['source']))

winteriscoming.add_child(cluster)

winteriscoming